In [109]:
import requests 
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
from bs4 import BeautifulSoup 
import time
from random import randint

In [2]:
load_dotenv()

True

In [3]:
linkedin = os.getenv("link")

In [4]:
params = {'oauth2_access_token': linkedin}
response = requests.get('https://api.linkedin.com/v2/me', params = params)

print(json.dumps(response.json(), indent=1))

{
 "localizedLastName": "Zapico",
 "profilePicture": {
  "displayImage": "urn:li:digitalmediaAsset:C4D03AQHQoc1REtj9Kw"
 },
 "firstName": {
  "localized": {
   "en_US": "Fernando"
  },
  "preferredLocale": {
   "country": "US",
   "language": "en"
  }
 },
 "lastName": {
  "localized": {
   "en_US": "Zapico"
  },
  "preferredLocale": {
   "country": "US",
   "language": "en"
  }
 },
 "id": "7_XX6j5GBR",
 "localizedFirstName": "Fernando"
}


In [5]:
response

<Response [200]>

In [6]:
datos = response.json()

In [7]:
datos

{'localizedLastName': 'Zapico',
 'profilePicture': {'displayImage': 'urn:li:digitalmediaAsset:C4D03AQHQoc1REtj9Kw'},
 'firstName': {'localized': {'en_US': 'Fernando'},
  'preferredLocale': {'country': 'US', 'language': 'en'}},
 'lastName': {'localized': {'en_US': 'Zapico'},
  'preferredLocale': {'country': 'US', 'language': 'en'}},
 'id': '7_XX6j5GBR',
 'localizedFirstName': 'Fernando'}

In [8]:
data = pd.read_csv('../data/Connections.csv', skiprows=2)


In [9]:
data.head()

,First Name,Last Name,Email Address,Company,Position,Connected On
0,María,Pérez de Ayala Satrústegui,NaN,The Kraft Heinz Company,Jr Brand Manager for Sauces,06 Nov 2021
1,Juan,Vega-Penichet Aguirre,NaN,Cabify,Global Growth Senior Specialist,06 Nov 2021
2,Silvia,Rico Sanz,NaN,Deoleo,Jr. Brand Manager,06 Nov 2021
3,Beatriz,Sánchez Portillo,NaN,Savills Aguirre Newman,Retail Building Consultant. Project Manager/Co...,05 Nov 2021
4,Daniel,Helguera Sanchez,NaN,Magna International,Senior Business Development,05 Nov 2021


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   First Name     771 non-null    object
 1   Last Name      771 non-null    object
 2   Email Address  18 non-null     object
 3   Company        750 non-null    object
 4   Position       753 non-null    object
 5   Connected On   776 non-null    object
dtypes: object(6)
memory usage: 36.5+ KB


In [11]:
def capastrings(x):
    """ 
    Funcion para limitar la informacion de los String a la mitad.
    argg = string
    """
    s = len(x)//2 
    return x[:s]

In [12]:
data["Name"] = data["First Name"].astype(dtype=str).apply(capastrings)

In [13]:
data["Lname"] = data["Last Name"].astype(dtype=str).apply(capastrings)

In [14]:
data = data.drop(['First Name', 'Last Name', 'Email Address'], axis = 1)

In [15]:
nombres_emp = list(data.Company)

In [16]:
print(type(nombres_emp))

<class 'list'>


In [17]:
nombres_emp = [str(nom).replace(" ", "+") for  nom in nombres_emp]
                    

In [18]:
data['nomb_busc'] = nombres_emp

# Funcion Sopera

In [19]:
# Lista de resultados a buscar 
data['urls'] = ['https://www.axesor.es/buscar/empresas?q='+comp+'#resultados' for comp in nombres_emp] 
data

,Company,Position,Connected On,Name,Lname,nomb_busc,urls
0,The Kraft Heinz Company,Jr Brand Manager for Sauces,06 Nov 2021,Ma,Pérez de Ayal,The+Kraft+Heinz+Company,https://www.axesor.es/buscar/empresas?q=The+Kr...
1,Cabify,Global Growth Senior Specialist,06 Nov 2021,Ju,Vega-Penic,Cabify,https://www.axesor.es/buscar/empresas?q=Cabify...
2,Deoleo,Jr. Brand Manager,06 Nov 2021,Sil,Rico,Deoleo,https://www.axesor.es/buscar/empresas?q=Deoleo...
3,Savills Aguirre Newman,Retail Building Consultant. Project Manager/Co...,05 Nov 2021,Bea,Sánchez,Savills+Aguirre+Newman,https://www.axesor.es/buscar/empresas?q=Savill...
4,Magna International,Senior Business Development,05 Nov 2021,Dan,Helguera,Magna+International,https://www.axesor.es/buscar/empresas?q=Magna+...
...,...,...,...,...,...,...,...
771,Universidad Politécnica de Madrid,Creación de Empresas UPM,29 Jul 2010,actú,Programa de Emp,Universidad+Politécnica+de+Madrid,https://www.axesor.es/buscar/empresas?q=Univer...
772,Finning South America,Gerente de Comunicaciones Corporativas en Finn...,19 Jul 2010,Dan,Flores,Finning+South+America,https://www.axesor.es/buscar/empresas?q=Finnin...
773,Colección SOLO,General Manager Espacio Solo,18 Jul 2010,El,de Andr,Colección+SOLO,https://www.axesor.es/buscar/empresas?q=Colecc...
774,ALBORAN CORPORATE FINANCE & STRATEGY,Responsable área de IT,06 Jul 2010,Pe,Garcia Fdez,ALBORAN+CORPORATE+FINANCE+&+STRATEGY,https://www.axesor.es/buscar/empresas?q=ALBORA...


In [33]:
def sopera(url):
    t=0
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.content,"html.parser")
        tabla = soup.findAll('table')
        elemento = tabla[0].find_all('a')[0]
        suburl = 'https:'+elemento['href']
        time.sleep(0.5)
        print(f'Rico Rico!{t}')
        t+=1
        return suburl
    except:
        return 'error'

In [32]:
data["suburl"] = data.urls.apply(sopera)

Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico

Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico Rico!1
Rico

In [34]:
data

,Company,Position,Connected On,Name,Lname,nomb_busc,urls,suburl
0,The Kraft Heinz Company,Jr Brand Manager for Sauces,06 Nov 2021,Ma,Pérez de Ayal,The+Kraft+Heinz+Company,https://www.axesor.es/buscar/empresas?q=The+Kr...,https://www.axesor.es/Informes-Empresas/990913...
1,Cabify,Global Growth Senior Specialist,06 Nov 2021,Ju,Vega-Penic,Cabify,https://www.axesor.es/buscar/empresas?q=Cabify...,https://www.axesor.es/Informes-Empresas/689260...
2,Deoleo,Jr. Brand Manager,06 Nov 2021,Sil,Rico,Deoleo,https://www.axesor.es/buscar/empresas?q=Deoleo...,https://www.axesor.es/Informes-Empresas/1766/D...
3,Savills Aguirre Newman,Retail Building Consultant. Project Manager/Co...,05 Nov 2021,Bea,Sánchez,Savills+Aguirre+Newman,https://www.axesor.es/buscar/empresas?q=Savill...,https://www.axesor.es/Informes-Empresas/117061...
4,Magna International,Senior Business Development,05 Nov 2021,Dan,Helguera,Magna+International,https://www.axesor.es/buscar/empresas?q=Magna+...,https://www.axesor.es/Informes-Empresas/774841...
...,...,...,...,...,...,...,...,...
771,Universidad Politécnica de Madrid,Creación de Empresas UPM,29 Jul 2010,actú,Programa de Emp,Universidad+Politécnica+de+Madrid,https://www.axesor.es/buscar/empresas?q=Univer...,https://www.axesor.es/Informes-Empresas/n/5217...
772,Finning South America,Gerente de Comunicaciones Corporativas en Finn...,19 Jul 2010,Dan,Flores,Finning+South+America,https://www.axesor.es/buscar/empresas?q=Finnin...,https://www.axesor.es/Informes-Empresas/960281...
773,Colección SOLO,General Manager Espacio Solo,18 Jul 2010,El,de Andr,Colección+SOLO,https://www.axesor.es/buscar/empresas?q=Colecc...,https://www.axesor.es/Informes-Empresas/327046...
774,ALBORAN CORPORATE FINANCE & STRATEGY,Responsable área de IT,06 Jul 2010,Pe,Garcia Fdez,ALBORAN+CORPORATE+FINANCE+&+STRATEGY,https://www.axesor.es/buscar/empresas?q=ALBORA...,https://www.axesor.es/Informes-Empresas/693640...


In [35]:
#data_primer_scra = data.to_csv('../data/data_primer_scra2.csv')

In [133]:
df = pd.read_csv('../data/data_primer_scra.csv')
df

,Unnamed: 0,Company,Position,Connected On,Name,Lname,nomb_busc,urls,suburl
0,0,The Kraft Heinz Company,Jr Brand Manager for Sauces,06 Nov 2021,Ma,Pérez de Ayal,The+Kraft+Heinz+Company,https://www.axesor.es/buscar/empresas?q=The+Kr...,https://www.axesor.es/Informes-Empresas/990913...
1,1,Cabify,Global Growth Senior Specialist,06 Nov 2021,Ju,Vega-Penic,Cabify,https://www.axesor.es/buscar/empresas?q=Cabify...,https://www.axesor.es/Informes-Empresas/689260...
2,2,Deoleo,Jr. Brand Manager,06 Nov 2021,Sil,Rico,Deoleo,https://www.axesor.es/buscar/empresas?q=Deoleo...,https://www.axesor.es/Informes-Empresas/1766/D...
3,3,Savills Aguirre Newman,Retail Building Consultant. Project Manager/Co...,05 Nov 2021,Bea,Sánchez,Savills+Aguirre+Newman,https://www.axesor.es/buscar/empresas?q=Savill...,https://www.axesor.es/Informes-Empresas/117061...
4,4,Magna International,Senior Business Development,05 Nov 2021,Dan,Helguera,Magna+International,https://www.axesor.es/buscar/empresas?q=Magna+...,https://www.axesor.es/Informes-Empresas/774841...
...,...,...,...,...,...,...,...,...,...
771,771,Universidad Politécnica de Madrid,Creación de Empresas UPM,29 Jul 2010,actú,Programa de Emp,Universidad+Politécnica+de+Madrid,https://www.axesor.es/buscar/empresas?q=Univer...,https://www.axesor.es/Informes-Empresas/n/5217...
772,772,Finning South America,Gerente de Comunicaciones Corporativas en Finn...,19 Jul 2010,Dan,Flores,Finning+South+America,https://www.axesor.es/buscar/empresas?q=Finnin...,https://www.axesor.es/Informes-Empresas/960281...
773,773,Colección SOLO,General Manager Espacio Solo,18 Jul 2010,El,de Andr,Colección+SOLO,https://www.axesor.es/buscar/empresas?q=Colecc...,https://www.axesor.es/Informes-Empresas/327046...
774,774,ALBORAN CORPORATE FINANCE & STRATEGY,Responsable área de IT,06 Jul 2010,Pe,Garcia Fdez,ALBORAN+CORPORATE+FINANCE+&+STRATEGY,https://www.axesor.es/buscar/empresas?q=ALBORA...,https://www.axesor.es/Informes-Empresas/693640...


In [149]:
list_of_dfs = [df.loc[i:i+10-1,:] for i in range(0, len(df),10)]

In [151]:
list_of_dfs[0]

,Unnamed: 0,Company,Position,Connected On,Name,Lname,nomb_busc,urls,suburl
0,0,The Kraft Heinz Company,Jr Brand Manager for Sauces,06 Nov 2021,Ma,Pérez de Ayal,The+Kraft+Heinz+Company,https://www.axesor.es/buscar/empresas?q=The+Kr...,https://www.axesor.es/Informes-Empresas/990913...
1,1,Cabify,Global Growth Senior Specialist,06 Nov 2021,Ju,Vega-Penic,Cabify,https://www.axesor.es/buscar/empresas?q=Cabify...,https://www.axesor.es/Informes-Empresas/689260...
2,2,Deoleo,Jr. Brand Manager,06 Nov 2021,Sil,Rico,Deoleo,https://www.axesor.es/buscar/empresas?q=Deoleo...,https://www.axesor.es/Informes-Empresas/1766/D...
3,3,Savills Aguirre Newman,Retail Building Consultant. Project Manager/Co...,05 Nov 2021,Bea,Sánchez,Savills+Aguirre+Newman,https://www.axesor.es/buscar/empresas?q=Savill...,https://www.axesor.es/Informes-Empresas/117061...
4,4,Magna International,Senior Business Development,05 Nov 2021,Dan,Helguera,Magna+International,https://www.axesor.es/buscar/empresas?q=Magna+...,https://www.axesor.es/Informes-Empresas/774841...
5,5,ING,Operations Analyst,05 Nov 2021,Lu,Ser,ING,https://www.axesor.es/buscar/empresas?q=ING#re...,https://www.axesor.es/Informes-Empresas/318046...
6,6,Olympic Channel,Data Coordinator,05 Nov 2021,Dan,Ar,Olympic+Channel,https://www.axesor.es/buscar/empresas?q=Olympi...,https://www.axesor.es/Informes-Empresas/837943...
7,7,Abbott Diagnostics Business,Technical Support Specialist Engineer FSE,04 Nov 2021,Car,de Leo,Abbott+Diagnostics+Business,https://www.axesor.es/buscar/empresas?q=Abbott...,https://www.axesor.es/Informes-Empresas/687428...
8,8,Librebit,Técnico,04 Nov 2021,Pe,Martín,Librebit,https://www.axesor.es/buscar/empresas?q=Libreb...,https://www.axesor.es/Informes-Empresas/314486...
9,9,"Guizhou Sinodrills Equipment Co.,LTD.",representante de ventas,03 Nov 2021,Oli,W,"Guizhou+Sinodrills+Equipment+Co.,LTD.",https://www.axesor.es/buscar/empresas?q=Guizho...,https://www.axesor.es/Informes-Empresas/429711...


In [152]:
len(list_of_dfs)

78

In [127]:
def avecrema(suburl):
    #print('1', suburl)
    t= 0
    try:
        dic_emp = {}
        loop=0
        r = requests.get(suburl)
        subsoup = BeautifulSoup(r.content,"html.parser")
        subtabla = subsoup.findAll('table')
        for i in range(9):
            subelemento = subtabla[0].find_all('tr')[i]
            t = subelemento.text
            tt = t.split(sep=':', maxsplit=-1)    
            dic_emp.update({tt[0]:tt[1]}) 
        #time.sleep(randint(2, 5))
        print(f'Rico Rico y enriquecido!  {loop}')
        loop += 1
        return dic_emp
    except:
        try:
            dic_emp = {}
            r = requests.get(suburl)
            subsoup = BeautifulSoup(r.content,"html.parser")
            subtabla = subsoup.findAll('table')
            itera = len(subtabla[0].find_all('tr'))
            for i in range(itera):
                subelemento = subtabla[0].find_all('tr')[i]
                t = subelemento.text
                tt = t.split(sep=':', maxsplit=-1)    
                dic_emp.update({tt[0]:tt[1]}) 
            #time.sleep(randint(2, 5))
            print(f'Rico Rico y enriquecido!  {loop}')
            loop += 1
            return dic_emp
        except:
            print('Error')
            return 'error'

In [153]:
su0 = list_of_dfs[0].suburl.apply(avecrema)

Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Error
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0


In [154]:
su0

0    {'
Nombre': '
The Keg Company Sociedad Limitad...
1    {'
Nombre': '
Cabify España sl', 'Dirección': ...
2    {'
Nombre': '
Deoleo sa', 'Dirección': 'C/ MAR...
3    {'
Nombre': '
Savills Aguirre Newman sa', 'Dir...
4    {'
Nombre': '
Magna Marbella International sl'...
5    {'
Nombre': '
Ingesport Health And Spa Consult...
6                                                error
7    {'
Nombre': '
Diagnostik Business sl', 'Direcc...
8    {'
Nombre': '
Control Suprimir sl', 'Dirección...
9    {'
Nombre': '
Drilling Equipment sl', 'CIF': '...
Name: suburl, dtype: object

In [155]:
su1 = list_of_dfs[1].suburl.apply(avecrema)

Error
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0


In [ ]:
su2 = list_of_dfs[2].suburl.apply(avecrema)

In [164]:
for i in range(len(list_of_dfs)):
    s = 'su'+str(i)
    s = list_of_dfs[i].suburl.apply(avecrema)
    time.sleep(randint(2, 5))

Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Error
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Rico Rico y enriquecido!  0
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Erro

In [22]:
#url = 'https://www.axesor.es/buscar/empresas?q='+'Deoleo'+'#resultados'
#r = requests.post(url, data='Deoleo')


Busqueda de cada empresa

In [23]:
#soup = BeautifulSoup(r.content,"html.parser")
#soup

In [24]:
#tabla = soup.findAll('table')
#tabla

In [25]:
#elemento = tabla[0].find_all('a')[0]
#elemento

In [26]:
#elemento.attrs

In [27]:
#suburl = 'https:'+elemento['href']

Entro en la pagina con info de la empresa

In [115]:
#salvo_df= df
#salvo_df.to_csv('../data/segScra.csv')

In [165]:
#df.suburl[12]

In [112]:
#subtabla


In [166]:
#subelemento = subtabla[0].find_all('tr')[0]
#subelemento

In [167]:
#t = subelemento.text
#t

In [60]:
#tt = t.split(sep=':', maxsplit=-1)
#tt

['\nNombre', '\nNuestra Bodega sl']

In [132]:
#t = 'Dirección:C/ MARIE CURIE, 7, 4ª PLTA.EDIFICIO BETA  28521, RIVAS-VACIAMADRID, MADRID   Ver mapa  Consultar dirección de las delegaciones '

#col, value = t.split(sep=':', maxsplit=-1)
#col
#value


'C/ MARIE CURIE, 7, 4ª PLTA.EDIFICIO BETA  28521, RIVAS-VACIAMADRID, MADRID   Ver mapa  Consultar dirección de las delegaciones '

In [168]:

#tt = (t.split(sep=':', maxsplit=1))
#dictt = {tt[0]:tt[1]}
#dictt


{'SIC': ' 6519 PROPIETARIOS-ADMINISTRADORES DE BIENES RAICES SC'}

In [135]:
#df = pd.DataFrame()
#df

""


In [124]:
#r = requests.get(suburl)
#print(r)
#print(r.headers)
#print(r.content)

In [125]:
#subsoup = BeautifulSoup(r.content,"html.parser")
#subsoup

In [126]:
#subtabla = subsoup.findAll('table')
#subtabla

In [125]:
#subelemento = subtabla[0].find_all('tr')[0]
#subelemento

<tr><td>
Nombre:
</td><td><h3 class="name">Deoleo sa</h3></td></tr>

In [126]:
#type(subelemento)

bs4.element.Tag

In [127]:
#subelemento.text

'\nNombre:\nDeoleo sa'

In [105]:
#subelemento.contents

[<td>
 Nombre:
 </td>,
 <td><h3 class="name">Deoleo sa</h3></td>]

In [124]:
#subelemento.text

'25% descuento en informe axesor 360 de esta empresa \xa0\xa0Acceder'

Para guardar en un df

In [108]:
#subelemento = subtabla[0].find_all('tr')[1]

In [109]:
#subelemento = 

<tr><td id="Direccion">Dirección:</td><td><span><span>C/ MARIE CURIE, 7, 4ª PLTA.EDIFICIO BETA</span> <span> 28521,</span> <span>RIVAS-VACIAMADRID, </span><span>MADRID</span> </span> <a class="link_target_blank not_print" href="https://maps.google.es/maps?q=(40.33909,-3.524620000000027)&amp;hl=es&amp;sll=40.33909,-3.524620000000027&amp;t=m&amp;z=17" id="link_google_maps" target="_blank" title="Localización de la empresa Deoleo sa"> <img alt="Localización de la empresa Deoleo sa" src="//recursos.axesor.org/img/map_fichas_1.png"/><span>Ver mapa</span> </a> <a class="not_print" data-productid="1766-1070" href="//www.axesor.es/ccs/comprar/servicio/empresa/38/1766?origC=FichaEmpresa-TablaInformacionGeneralDelegaciones" id="verDelegaciones" rel="nofollow" title="Otras direcciones de Deoleo sa">Consultar dirección de las delegaciones</a> </td></tr>

In [110]:
#subelemento.text

'Dirección:C/ MARIE CURIE, 7, 4ª PLTA.EDIFICIO BETA  28521, RIVAS-VACIAMADRID, MADRID   Ver mapa  Consultar dirección de las delegaciones '

In [111]:
#subelemento = subtabla[0].find_all('tr')[2]

In [112]:
#subelemento.text

'Teléfono: 915589505 \xa0 Consultar si la empresa tiene otros teléfonos '

In [113]:
#subelemento = subtabla[0].find_all('tr')[3]

In [114]:
#subelemento.text

'CIF:A48012009'

In [115]:
#subelemento = subtabla[0].find_all('tr')[4]
#subelemento.text

'Sitio Web:www.deoleo.com'

In [116]:
#subelemento = subtabla[0].find_all('tr')[5]
#subelemento.text

'Email:atencion.cliente@deoleo.com'

In [117]:
#subelemento = subtabla[0].find_all('tr')[6]
#subelemento.text

'Forma jurídica:SOCIEDAD ANONIMA                        '

In [118]:
#subelemento = subtabla[0].find_all('tr')[7]
#subelemento.text

'Constituida hace: 66 años, 8 meses y 26 dias'

In [119]:
#subelemento = subtabla[0].find_all('tr')[8]
#subelemento.text

'Objeto social:  COMPRAVENTA IMPORTACION EXPORTACION TRANSFORMACIÓN ELABORACION Y COMERCIALIZACION DE TODA CLASE DE GALLETAS OBLEAS BARQUILLOS  [+]  CARAMELOS Y PRODUCTOS SIMILARES. FABRICACION, INDUSTRIALIZACION COMPRA VENTA IMPORTACION Y EXPORTACION DE TODA CLASE DE ACEITES MARGARINAS GRASAS SUS DERIVADOS, SALSAS, CONDIMENTOS DE MESA Y CONSERVAS, ASI COMO DE CUALESQUIERA OTROS PRODUCTOS ALIMENTICIOS Y AGRICOLAS  '

In [120]:
#subelemento = subtabla[0].find_all('tr')[9]
#subelemento.text

'CNAE: 1043  Fabricación de aceite de oliva'

In [121]:
#subelemento = subtabla[0].find_all('tr')[10]
#subelemento.text

'SIC: 2079 Aceites de mesa, margarina y productos grasos y oleícolas comestibles'

In [14]:
#print(data.shape)

(776, 6)


In [15]:
#data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   First Name     771 non-null    object
 1   Last Name      771 non-null    object
 2   Email Address  18 non-null     object
 3   Company        750 non-null    object
 4   Position       753 non-null    object
 5   Connected On   776 non-null    object
dtypes: object(6)
memory usage: 36.5+ KB


In [16]:
#data = data.drop('Email Address', axis = 1)

In [17]:
#data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   First Name    771 non-null    object
 1   Last Name     771 non-null    object
 2   Company       750 non-null    object
 3   Position      753 non-null    object
 4   Connected On  776 non-null    object
dtypes: object(5)
memory usage: 30.4+ KB


In [18]:
#len(data.Position.unique())

634

In [19]:
#data.Position.value_counts()

Director                                  12
Gerente                                   10
CEO                                        9
Director General                           7
Director comercial                         6
                                          ..
Coordinador Burgos Film Commission         1
Ingeniero de Diseño y Transporte Naval     1
Socio Fundador y Director Gerente          1
Logopeda en Unidad de Daño Cerebral        1
SEO Consultant                             1
Name: Position, Length: 633, dtype: int64

In [170]:
import plotly.express as px

fig = px.scatter(df, x="First Name", y="Connected On")
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['Unnamed: 0', 'Company', 'Position', 'Connected On', 'Name', 'Lname', 'nomb_busc', 'urls', 'suburl'] but received: First Name

In [ ]:
fig = px.treemap(data, path=['Position', 'Company'], width=1200, height=1200)

fig.show()

In [173]:
!jupyter nbconvert --to script MiPerfil.ipynb

[NbConvertApp] Converting notebook MiPerfil.ipynb to script
[NbConvertApp] Writing 6958 bytes to MiPerfil.py
